In [1]:
# Injected parameters
max_depth_model = 3
max_number_estimator = 200


bi + ml --> final

In [2]:
import sys
sys.path.append('../../../')

In [3]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import FunctionTransformer

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
st.set_option('deprecation.showPyplotGlobalUse', False)

# Functions

In [4]:
def log_transform(x):
    return np.log(x + 1)

def model_training(X_train, y_train, numerical_cols, categorical_cols,parameters,multiclass):
    """
    
    """
    
    ##
    transformer = FunctionTransformer(log_transform)
    numerical_preprocessor = Pipeline(steps=[("imputer", IterativeImputer(ExtraTreesRegressor(n_estimators=5,random_state=1),random_state=1,verbose=0,add_indicator=True)),
                                     ("scaler", transformer)]) #MinMaxScaler()
    categorical_preprocessor = Pipeline(steps=[("imputer", SimpleImputer(strategy='constant', fill_value='missing',add_indicator=True)),
                                           ("label_enc", OneHotEncoder(handle_unknown='ignore'))])
    preprocessor = ColumnTransformer(transformers=[("numerical_preprocessor", numerical_preprocessor, numerical_cols),
                                               ("categorical_preprocessor", categorical_preprocessor, categorical_cols)])
    pipe_model = GradientBoostingClassifier(random_state=0,n_iter_no_change=25,warm_start=True,max_features=1.0)
    ##
    model = Pipeline(steps=[('preprocessor', preprocessor),('model', pipe_model)])
    
    ##
    # ('model', CalibratedClassifierCV(base_estimator=pipe_model,method='isotonic'))])
    ##
    if multiclass == 'no':
        model_grid = GridSearchCV(model,parameters,cv=4,scoring='recall',verbose=0,return_train_score=True).fit(X_train,y_train)
    else:
        model_grid = GridSearchCV(model,parameters,cv=4,scoring='accuracy',verbose=0,return_train_score=True).fit(X_train,y_train)
    st.markdown('##### GridSearchCV results...')
    st.write("Mean Train Scores: \n{}\n".format(model_grid.cv_results_['mean_train_score']))
    st.write("Mean CV Scores: \n{}\n".format(model_grid.cv_results_['mean_test_score']))
    st.write("Best Parameters: \n{}\n".format(model_grid.best_params_))
    
    return model_grid

In [5]:
def model_calibration(model,X_test,y_test):
    """
    
    """
    st.markdown("##### Calibrating the model...")
    best_model = model.best_estimator_
    calibrator = CalibratedClassifierCV(best_model, cv='prefit')
    calibrator = calibrator.fit(X_test,y_test)
    return calibrator

In [6]:
def model_evaluation(model,X_test,y_test):
    """
    
    """
    st.markdown('##### Test results...')
    y_test_predict_grid = model.predict(X_test)    
    st.write("Model Test Recall:", metrics.recall_score(y_test, y_test_predict_grid))
    st.write('--------------------------------------------------')
    st.write('Model Test Confusion Matrix')
    cm = confusion_matrix(y_test,y_test_predict_grid,normalize='true') 
    ConfusionMatrixDisplay(cm,display_labels=['No','Yes']).plot()
    col1, col2 = st.columns([1, 2])
    col1.pyplot()
    col2.text('Model Report:\n ' + classification_report(y_test, y_test_predict_grid))

    st.markdown("##### Features importance: ")
    feature_importances = model.named_steps['model'].feature_importances_
    feature_names = X_test.columns
    lista = []
    val = []
    for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
#             st.write('{}: {}'.format(name, score))
            lista.append(name)
            val.append(score)
    ris = pd.DataFrame({'feature': lista, 'score':val})
    st.plotly_chart(px.bar(ris,x='feature',y='score'))
    return model

# 1.0 Data retrieval

In [7]:
df = pd.read_pickle("app/datalake/train_eng.pkl")
val = pd.read_pickle("app/datalake/val_eng.pkl")

In [8]:
cols = ['card1', 'card2', 'card3','card5','M4', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25',
       'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_32', 'id_33', 'id_34',
       'id_35', 'id_36', 'id_37', 'id_38','num_transaction_per_time']

In [9]:
df = df.drop(columns=cols)

In [10]:
val = val.drop(columns=cols)

# 2.0 Machine Learning
## 2.1 Training model

In [11]:
for i in df:
    if df[i].dtypes == 'object':
        df[i] = df[i].astype(str)

In [12]:
X_train = df.drop(columns='isFraud')
y_train = df['isFraud'].astype(float)

In [13]:
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'bool']).columns

In [14]:
max_depth_model = 3
max_n_est = 200

In [15]:
parameters = {'model__max_depth':[2,max_depth_model],'model__n_estimators':[100,max_n_est],'model__learning_rate':[0.1],'model__min_samples_leaf':[3]}

In [16]:
model = model_training(X_train,y_train,numerical_cols,categorical_cols,parameters,'no')

## 2.2 Evaluation model

In [17]:
for i in val:
    if val[i].dtypes == 'object':
        val[i] = val[i].astype(str)

In [18]:
X_val = val.drop(columns='isFraud')
y_val = val['isFraud'].astype(float)

In [19]:
m = model_calibration(model,X_val,y_val)

In [20]:
model_evaluation(m.base_estimator, X_val, y_val)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical_preprocessor',
                                                  Pipeline(steps=[('imputer',
                                                                   IterativeImputer(add_indicator=True,
                                                                                    estimator=ExtraTreesRegressor(n_estimators=5,
                                                                                                                  random_state=1),
                                                                                    random_state=1)),
                                                                  ('scaler',
                                                                   FunctionTransformer(func=<function log_transform at 0x7f56ceeacb80>))]),
                                                  Index(['TransactionAmt', 'max_c', 'max_d',...
                                 

<Figure size 432x288 with 0 Axes>

# 6.0 Model Output

In [21]:
with open('app/datalake/trained_model_v1.pkl','wb') as file:
    pickle.dump(m.base_estimator, file)